In [ ]:
!pip uninstall -y torch torchvision torchaudio transformers accelerate peft bitsandbytes
!pip install torch==2.2.2 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers==4.41.2 peft==0.10.0 accelerate bitsandbytes==0.46.1 sentencepiece datasets
!pip install numpy==1.26.4 --force-reinstall
import os
os.kill(os.getpid(), 9)

Found existing installation: torch 2.2.2+cu118
Uninstalling torch-2.2.2+cu118:
  Successfully uninstalled torch-2.2.2+cu118
Found existing installation: torchvision 0.17.2+cu118
Uninstalling torchvision-0.17.2+cu118:
  Successfully uninstalled torchvision-0.17.2+cu118
Found existing installation: torchaudio 2.2.2+cu118
Uninstalling torchaudio-2.2.2+cu118:
  Successfully uninstalled torchaudio-2.2.2+cu118
Found existing installation: transformers 4.41.2
Uninstalling transformers-4.41.2:
  Successfully uninstalled transformers-4.41.2
Found existing installation: accelerate 1.9.0
Uninstalling accelerate-1.9.0:
  Successfully uninstalled accelerate-1.9.0
Found existing installation: peft 0.16.0
Uninstalling peft-0.16.0:
  Successfully uninstalled peft-0.16.0
Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.2.2%2Bcu118-cp311-cp311-linux_x86_64.whl (819.2 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchvis

  Using cached transformers-4.41.2-py3-none-any.whl.metadata (43 kB)
Using cached transformers-4.41.2-py3-none-any.whl (9.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.1/367.1 kB 31.2 MB/s eta 0:00:00
ERROR: Operation cancelled by user
^C


In [ ]:
"""
Feinabstimmung von gbert-large für Stance Detection mit klassen-gewichteter Verlustfunktion.
Bewertet Macro-F1 auf dem Validierungsdatensatz.
"""

# Bibliotheken importieren
import logging, inspect
from pathlib import Path
from collections import Counter

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from datasets import Dataset
from sklearn.metrics import accuracy_score, f1_score
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Konfiguration
DATA_PATH = Path("/content/")
MODEL_NAME = "deepset/gbert-large"
SAVE_DIR = Path("/content/Output/gbert-large-finetuned")
SAVE_DIR.mkdir(parents=True, exist_ok=True)

label2id = {"Zustimmung": 0, "Neutral": 1, "Ablehnung": 2}
id2label = {v: k for k, v in label2id.items()}

# CSV-Ladefunktion
def load_flat_dataset(fname):
    df = pd.read_csv(DATA_PATH / fname, sep=";", encoding="utf-8-sig").dropna()

    if "\ufefftext" in df.columns:
        df = df.rename(columns={"﻿text": "text"})

    def flatten(val):
        if isinstance(val, list):
            return " ".join(map(str, val))
        return str(val)

    df["text"] = df["text"].apply(flatten)
    df["label"] = df["label"].map(label2id)

    print(f"{fname} geladen: {len(df)} Zeilen")
    print("Beispiel:", df.iloc[0]["text"])
    print("Typen:", df["text"].apply(type).value_counts())
    return Dataset.from_pandas(df)

# Tokenizer und Data Collator
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tok_fn(batch):
    return tokenizer(batch["text"], truncation=True, padding=True, max_length=256)

collator = DataCollatorWithPadding(tokenizer)

# Trainings- und Validierungsdaten laden
train_ds = load_flat_dataset("train.csv")
val_ds   = load_flat_dataset("val.csv")

train_ds = train_ds.map(tok_fn, batched=True)
val_ds   = val_ds.map(tok_fn, batched=True)

# Modell mit klassen-gewichteter Verlustfunktion
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=3, id2label=id2label, label2id=label2id
)

counts  = Counter(train_ds["label"])
N_total = sum(counts.values())
weights = torch.tensor([N_total / (counts[i] * 3) for i in range(3)], dtype=torch.float)
logger.info("Klassenverteilung %s → Gewichte %s", counts, weights)

# Trainingsargumente
kw = "evaluation_strategy" if "evaluation_strategy" in inspect.signature(TrainingArguments.__init__).parameters else "eval_strategy"

training_args = TrainingArguments(
    **{
        "output_dir": str(SAVE_DIR),
        kw: "epoch",
        "save_strategy": "epoch",
        "logging_strategy": "steps",
        "logging_steps": 50,
        "learning_rate": 2e-5,
        "per_device_train_batch_size": 8,
        "per_device_eval_batch_size": 8,
        "gradient_accumulation_steps": 4,
        "num_train_epochs": 5,
        "weight_decay": 0.1,
        "warmup_ratio": 0.1,
        "fp16": True,
        "load_best_model_at_end": True,
        "metric_for_best_model": "eval_macro_f1",
        "logging_dir": str(SAVE_DIR / "logs"),
        "report_to": "none",
    }
)

# Metriken
def metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "macro_f1": f1_score(labels, preds, average="macro"),
    }

# Trainer mit klassen-gewichteter Loss
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels").to(model.device)
        outputs = model(**inputs)
        weight_vec = weights.to(outputs.logits.device)
        loss = nn.functional.cross_entropy(outputs.logits, labels, weight=weight_vec)
        return (loss, outputs) if return_outputs else loss

# Training starten
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=collator,
    compute_metrics=metrics,
)

trainer.train()

# Bestes Modell speichern
trainer.save_model(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)
logger.info("Bestes Modell gespeichert in %s", SAVE_DIR)


📄 train.csv geladen: 3150 Zeilen
🔍 Beispiel: Ich kenne mehrere, große und langjährige Papierfabriken, die von Ausländischen Investoren aufgekauft wurden und nun alle Abgewirtschaftet und geschlossen sind. Im besten Deutschland aller Zeiten.
🧪 Typen: text
<class 'str'>    3150
Name: count, dtype: int64
📄 val.csv geladen: 675 Zeilen
🔍 Beispiel: Der Ball ist rund und ein Spiel dauert 90 Minuten.
🧪 Typen: text
<class 'str'>    675
Name: count, dtype: int64


Map:   0%|          | 0/3150 [00:00<?, ? examples/s]

Map:   0%|          | 0/675 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at deepset/gbert-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1889146330.py:125: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,1.048900,0.735065,0.687407,0.664649
2,0.640100,0.697799,0.739259,0.693130
3,0.441500,0.754644,0.745185,0.708895
4,0.260000,0.854552,0.761481,0.720637
5,0.132700,1.020976,0.767407,0.725125


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
from huggingface_hub import login, create_repo
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from pathlib import Path

# Lade den Hugging Face Token aus Colab Secrets
from google.colab import userdata
HF_TOKEN = userdata.get("HF_TOKEN")  # Vorher in den Secrets setzen

if HF_TOKEN is None:
    raise ValueError("Hugging Face Token nicht gefunden. Bitte unter 'Notebook > Secrets' als 'HF_TOKEN' hinzufügen.")

# Lokaler Modellpfad und Ziel-Repository
MODEL_DIR = Path("/content/Output/gbert-large-finetuned")
REPO_NAME = "gbert-large-stance-finetuned"
HF_USERNAME = "YangZexi"
REPO_ID = f"{HF_USERNAME}/{REPO_NAME}"

# Anmeldung und Repository anlegen
login(token=HF_TOKEN)
create_repo(repo_id=REPO_ID, exist_ok=True)

# Modell und Tokenizer laden
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR)
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)

# Modell und Tokenizer zu Hugging Face hochladen
model.push_to_hub(REPO_ID)
tokenizer.push_to_hub(REPO_ID)

print(f"Erfolgreich hochgeladen: https://huggingface.co/{REPO_ID}")


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmplr6qycr4/model.safetensors    :   0%|          | 44.0kB / 1.34GB            

README.md: 0.00B [00:00, ?B/s]

✅ Erfolgreich hochgeladen: https://huggingface.co/YangZexi/gbert-large-stance-finetuned


In [ ]:
# Evaluation-Skript – GBERT-Large (Klassifikator)
import torch
import pandas as pd
import numpy as np
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import classification_report

# Pfade
MODEL_DIR = Path("/content/Output/gbert-large-finetuned")  # ggf. anpassen
TEST_PATH = Path("/content/test.csv")
LABEL_MAP = {"Zustimmung": 0, "Neutral": 1, "Ablehnung": 2}
ID2LABEL  = {v: k for k, v in LABEL_MAP.items()}

# Modell und Tokenizer laden
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR)
model.eval().to("cuda" if torch.cuda.is_available() else "cpu")

# Testdaten laden
df = pd.read_csv(TEST_PATH, sep=";", encoding="utf-8-sig")
df = df[df["label"].isin(LABEL_MAP.keys())].copy()
df["label_id"] = df["label"].map(LABEL_MAP)

# Vorhersagefunktion
def predict(texts, batch_size=16):
    preds = []
    device = next(model.parameters()).device
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        enc = tokenizer(
            batch.tolist(),
            padding=True,
            truncation=True,
            max_length=256,
            return_tensors="pt"
        ).to(device)
        with torch.no_grad():
            logits = model(**enc).logits
        pred_ids = torch.argmax(logits, dim=-1).cpu().numpy()
        preds.extend(pred_ids)
    return preds

# Vorhersagen berechnen
df["pred_id"] = predict(df["text"])
df["pred"] = df["pred_id"].map(ID2LABEL)

# Klassifikationsbericht ausgeben
print("\nKlassifikationsbericht:")
print(classification_report(df["label"], df["pred"], labels=list(LABEL_MAP.keys())))

# Ergebnisse speichern
OUT_PATH = MODEL_DIR.parent / f"{MODEL_DIR.name}_predictions.csv"
df.to_csv(OUT_PATH, sep=";", index=False, encoding="utf-8-sig")
print(f"\nVorhersagen gespeichert unter: {OUT_PATH}")


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



🔢 Classification Report:
              precision    recall  f1-score   support

  Zustimmung       0.67      0.64      0.66       149
     Neutral       0.57      0.57      0.57       143
   Ablehnung       0.82      0.83      0.83       383

    accuracy                           0.74       675
   macro avg       0.69      0.68      0.69       675
weighted avg       0.74      0.74      0.74       675


📂 Predictions saved to: /content/Output/gbert-large-finetuned_predictions.csv
